# 1. Pembuatan Pipeline ETL

In [6]:
import pandas as pd

# URL dataset
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0273EN-SkillsNetwork/labs/v1/m3/data/Project_data.csv"

# Membaca dataset ke dalam dataframe
df = pd.read_csv(url)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,01-12-2010 08:45,3.75,12583.0,France
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,01-12-2010 08:45,3.75,12583.0,France
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,01-12-2010 08:45,3.75,12583.0,France
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,01-12-2010 08:45,0.85,12583.0,France
4,536370,21883,STARS GIFT TAPE,24,01-12-2010 08:45,0.65,12583.0,France


In [7]:
# Menghapus baris dengan InvoiceNo yang dimulai dengan 'C'
df = df[~df['InvoiceNo'].str.startswith('C')]

# Menghapus baris dengan StockCode yang tidak diinginkan
unwanted_stock_codes = ['C2', 'D', 'M', 'POST']
df = df[~df['StockCode'].isin(unwanted_stock_codes)]

# Menghapus baris dengan CustomerID yang hilang
df = df.dropna(subset=['CustomerID'])

df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,01-12-2010 08:45,3.75,12583.0,France
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,01-12-2010 08:45,3.75,12583.0,France
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,01-12-2010 08:45,3.75,12583.0,France
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,01-12-2010 08:45,0.85,12583.0,France
4,536370,21883,STARS GIFT TAPE,24,01-12-2010 08:45,0.65,12583.0,France


In [14]:
import sqlite3

# Membuat koneksi ke database SQLite
conn = sqlite3.connect('Invoice_Records.db')

# Memuat dataframe ke tabel SQLite
df.to_sql('Purchase_transactions', conn, if_exists='replace', index=False)

# Menampilkan tabel yang ada di dalam database
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

                    name
0  Purchase_transactions


In [15]:
conn.close()

# 2. Query Database

In [19]:
# Membuka kembali koneksi ke database SQLite
conn = sqlite3.connect('Invoice_Records.db')

# Query SQL untuk mengambil data transaksi untuk Jerman
query = """
SELECT * FROM Purchase_transactions WHERE Country = 'Germany';
"""

# Menjalankan query dan menyimpan hasilnya ke dalam dataframe
germany_transactions = pd.read_sql(query, conn)
germany_transactions.head()




,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536527,22809,SET OF 6 T-LIGHTS SANTA,6,01-12-2010 13:04,2.95,12662.0,Germany
1,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,01-12-2010 13:04,2.55,12662.0,Germany
2,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,01-12-2010 13:04,0.85,12662.0,Germany
3,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,01-12-2010 13:04,1.65,12662.0,Germany
4,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,01-12-2010 13:04,1.95,12662.0,Germany


In [17]:
conn.close()

# 3. Analisis Data dan Penambangan Data

In [21]:
# Grupkan data berdasarkan InvoiceNo dan kumpulkan Descriptions ke dalam list
grouped_data = germany_transactions.groupby('InvoiceNo')['Description'].apply(list).tolist()

# One-hot encoding menggunakan TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(grouped_data).transform(grouped_data)

# Konversi hasil encoding ke DataFrame
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

df_encoded.head()




,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
# Menjalankan Apriori algorithm
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)

# Menentukan aturan asosiasi
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values('confidence', ascending=False).head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
15728,"(ROUND SNACK BOXES SET OF4 WOODLAND , ROUND SN...",(SPACEBOY CHILDRENS CUP),0.013544,0.045147,0.013544,1.0,22.150000,0.012933,inf,0.967963
8336,"(PACK OF 20 NAPKINS PANTRY DESIGN, SPACEBOY LU...",(ROUND SNACK BOXES SET OF4 WOODLAND ),0.011287,0.252822,0.011287,1.0,3.955357,0.008433,inf,0.755708
17137,"(LUNCH BAG WOODLAND, RED RETROSPOT CUP, RED RE...","(ROUND SNACK BOXES SET OF4 WOODLAND , JUMBO BA...",0.011287,0.038375,0.011287,1.0,26.058824,0.010854,inf,0.972603
3731,"(JUMBO STORAGE BAG SUKI, 6 RIBBONS RUSTIC CHARM)",(CHOCOLATE BOX RIBBONS ),0.011287,0.038375,0.011287,1.0,26.058824,0.010854,inf,0.972603
11111,"(GUMBALL COAT RACK, 6 RIBBONS RUSTIC CHARM, RE...",(SPACEBOY LUNCH BOX ),0.011287,0.106095,0.011287,1.0,9.425532,0.010089,inf,0.904110


In [23]:
rules.to_csv('association_rules.csv', index=False)
